# Eligibility Signposting API - Request Journey Demo

**A practical walkthrough showing how requests get processed in our API**

---

---

# The Journey of a Request

When a request arrives at our API, it follows this path:

```
API Gateway → AWS Lambda → Flask App → Presentation Layer → Business Logic Layer → Response
```

What actually happens?

1. Work out what is being asked for (which endpoint) and validate request looks ok - Presentation Layer
1. Pull in person data and campaign data - Data Access Layer
2. Set up an eligibility calculator and calculate eligibility - Business Logic Layer
3. Return response

First of all though, lets see an actual response.

In [35]:
# ruff: noqa
import requests
import json

url = "https://sandbox.api.service.nhs.uk/eligibility-signposting-api/patient-check/9686368973"
headers = {"apiKey": "test"}

response = requests.get(url, headers=headers)
print("Status Code:", response.status_code)
print()
print("Response Body:")
print("=" * 60)

# Try to parse and prettify JSON response
try:
    response_json = response.json()
    print(json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    # If not JSON, print as text
    print(response.text)

Status Code: 200

Response Body:
{
  "responseId": "1a233ba5-e1eb-4080-a086-2962f6fc3473",
  "meta": {
    "lastUpdated": "2025-02-12T16:11:22Z"
  },
  "processedSuggestions": [
    {
      "condition": "RSV",
      "status": "Actionable",
      "statusText": "You should have the RSV vaccine",
      "eligibilityCohorts": [
        {
          "cohortCode": "rsv_age_rolling",
          "cohortText": "are aged 75 to 79",
          "cohortStatus": "Actionable"
        }
      ],
      "suitabilityRules": [],
      "actions": [
        {
          "actionType": "InfoText",
          "actionCode": "BookLocal",
          "description": "## Get vaccinated at your GP surgery\n\nContact your GP surgery to book an appointment."
        },
        {
          "actionType": "ButtonWithAuthLinkWithInfoText",
          "actionCode": "BookNBSInfoText",
          "description": "## Book an appointment online at a pharmacy\n\nYou can book an appointment online at a pharmacy that offers the RSV vaccinat

In [34]:
# Example configuration which drives some of the above responses

# Rules drive which messages someone receives via 'Comms Routing' from 'R' (Routing) rules

rules = repr({
     "IterationRules": [
         {
            "Type": "R",
            "Name": "Within CP Expansion ICB not 80 plus",
            "Description": "Book an appointment on NBS as within CP expansion",
            "Priority": 1200,
            "Operator": "in",
            "Comparator": "QH8,QJG",
            "AttributeLevel": "PERSON",
            "AttributeName": "ICB",
            "CommsRouting": "CONTACT_GP|BOOK_NBS_INFO|WALKIN|HELP_SUPPORT"
          },
          {
            "Type": "R",
            "Name": "Within CP Expansion ICB not 80 plus",
            "Description": "Book an appointment on NBS as within CP expansion",
            "Priority": 1200,
            "AttributeLevel": "PERSON",
            "AttributeName": "DATE_OF_BIRTH",
            "Operator": "Y>",
            "Comparator": "-80",
            "CommsRouting": "CONTACT_GP|BOOK_NBS_INFO|WALKIN|HELP_SUPPORT"
          }
     ]
 }
)

# Content is defined for each of these routings and returned with the response
config = repr(
{
      "CONTACT_GP": {
        "ExternalRoutingCode": "ContactGP",
        "ActionDescription": "## Get vaccinated at your GP practice\n\nContact your GP surgery to book an appointment.",
        "ActionType": "InfoText",
        "UrlLink": None,
        "UrlLabel": ""
       },
      "BOOK_NBS_INFO": {
          "ExternalRoutingCode": "BookNBSInfoText",
          "ActionDescription": "## Book an appointment online at a pharmacy\n\nYou can book an appointment online at a pharmacy that offers the RSV vaccination. You need to be registered with a GP to do this.",
          "ActionType": "ButtonWithAuthLinkWithInfoText",
          "UrlLink": "https://f.nhswebsite-integration.nhs.uk/nbs/nhs-app/rsv",
          "UrlLabel": "Continue to booking"
        },
      "WALKIN": {
          "ExternalRoutingCode": "WalkIn",
          "ActionDescription": "## Get vaccinated without an appointment\n\nYou can get an RSV vaccination at some pharmacies without needing an appointment.\n\nYou do not need to be registered with a GP to do this.",
          "ActionType": "ActionLinkWithInfoText",
          "UrlLink": "https://www.nhs.uk/service-search/vaccination-and-booking-services/find-a-pharmacy-where-you-can-get-a-free-rsv-vaccination",
          "UrlLabel": "Find a pharmacy where you can get a free RSV vaccination"
      },
      "HELP_SUPPORT": {
          "ExternalRoutingCode": "HelpSupportInfo",
          "ActionDescription": "## If you think this is incorrect\n\nIf you have already had this vaccination or your personal details are wrong, visit our [help and support page](https://www.nhs.uk/nhs-app/nhs-app-help-and-support/).",
          "ActionType": "InfoText",
          "UrlLink": None,
          "UrlLabel": ""
      }
}
)

Responses will fall into three main categories

* Actionable - this is someone Eligible and Actionable
* Not Eligible
* Not Actionable - this is someone Eligible but not Actionable



## Demo - let's run through some requests and responses to explore these responses, and see how the message can be shaped in responses from our Sandbox environment

## Setup

First, let's set up our environment, initialize the Flask app, and import the necessary modules:

In [36]:
# Add the source directory to the Python path
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd() / 'src'))

# Initialize Flask app to provide application context for validators
from flask import Flask, g

app = Flask(__name__)
app_context = app.app_context()
app_context.push()

# Initialize audit log in Flask's g object (needed for eligibility calculator)
from eligibility_signposting_api.audit.audit_models import AuditEvent
g.audit_log = AuditEvent()

# Import the validation code
from eligibility_signposting_api.common.request_validator import (
    validate_query_params,
    validate_nhs_number
)

print("✓ Flask app initialized")
print("✓ Audit context initialized")
print("✓ Setup complete")

✓ Flask app initialized
✓ Audit context initialized
✓ Setup complete


---

# Part 1: Request Validation

Every request must pass validation before reaching our business logic.

## Example 1: A Valid Request ✓

In [37]:
print("Testing a VALID request:")
print("=" * 60)
print()

# Good request parameters
good_path_nhs = "9000000009"
good_header_nhs = "9000000009"
good_query_params = {
    "category": "VACCINATIONS",
    "conditions": "RSV",
    "includeActions": "Y"
}

print(f"NHS Number (path): {good_path_nhs}")
print(f"NHS Number (header): {good_header_nhs}")
print(f"Query Parameters: {good_query_params}")
print()

# Validate NHS number
nhs_valid = validate_nhs_number(good_path_nhs, good_header_nhs)
print(f"NHS Number validation: {'✓ PASS' if nhs_valid else '✗ FAIL'}")

# Validate query parameters
params_valid, error = validate_query_params(good_query_params)
print(f"Query params validation: {'✓ PASS' if params_valid else '✗ FAIL'}")

if nhs_valid and params_valid:
    print()
    print("✅ Request is valid - proceeding to eligibility check")
else:
    print(f"❌ Request failed: {error}")

Testing a VALID request:

NHS Number (path): 9000000009
NHS Number (header): 9000000009
Query Parameters: {'category': 'VACCINATIONS', 'conditions': 'RSV', 'includeActions': 'Y'}

NHS Number validation: ✓ PASS
Query params validation: ✓ PASS

✅ Request is valid - proceeding to eligibility check


## Example 2: Invalid Request - NHS Number Mismatch ✗

In [38]:
print("Testing INVALID request - NHS number mismatch:")
print("=" * 60)
print()

bad_path_nhs = "9000000009"
bad_header_nhs = "9000000010"  # Different!
query_params = {
    "category": "VACCINATIONS",
    "conditions": "RSV",
    "includeActions": "Y"
}

print(f"NHS Number (path): {bad_path_nhs}")
print(f"NHS Number (header): {bad_header_nhs}")
print()

nhs_valid = validate_nhs_number(bad_path_nhs, bad_header_nhs)
print(f"NHS Number validation: {'✓ PASS' if nhs_valid else '✗ FAIL'}")

if not nhs_valid:
    print()
    print("❌ HTTP 401 Unauthorized")
    print("Error: You are not authorised to request information for the supplied NHS Number")

NHS number mismatch


Testing INVALID request - NHS number mismatch:

NHS Number (path): 9000000009
NHS Number (header): 9000000010

NHS Number validation: ✗ FAIL

❌ HTTP 401 Unauthorized
Error: You are not authorised to request information for the supplied NHS Number


## Example 3: Invalid Request - Bad Query Parameters ✗

In [39]:
import json

print("Testing INVALID request - bad query parameters:")
print("=" * 60)
print()

bad_query_params = {
    "category": "INVALID_CATEGORY",  # Not VACCINATIONS, SCREENING, or ALL
    "conditions": "RSV",
    "includeActions": "Y"
}

print(f"Query Parameters: {bad_query_params}")
print()

params_valid, error_response = validate_query_params(bad_query_params)
print(f"Query params validation: {'✓ PASS' if params_valid else '✗ FAIL'}")

if not params_valid:
    print()
    print("❌ HTTP 400 Bad Request")
    print()
    # Parse the Flask Response object to show details
    # error_response is a Flask Response object (body, status_code, headers)
    response_body = error_response.get_data(as_text=True)
    error_body = json.loads(response_body)
    
    if "issue" in error_body and len(error_body["issue"]) > 0:
        issue = error_body["issue"][0]
        print(f"Error: {issue.get('diagnostics', 'Invalid parameters')}")
    print("Valid categories: VACCINATIONS, SCREENING, ALL")

Invalid category query param: 'INVALID_CATEGORY'


Testing INVALID request - bad query parameters:

Query Parameters: {'category': 'INVALID_CATEGORY', 'conditions': 'RSV', 'includeActions': 'Y'}

Query params validation: ✗ FAIL

❌ HTTP 400 Bad Request

Error: INVALID_CATEGORY is not a category that is supported by the API
Valid categories: VACCINATIONS, SCREENING, ALL


---

# Actionable response

Now let's create the data for our first scenario:
- **Person**: 76 years old, no RSV vaccination, Leeds area, RSV cohort member

## Creating a Person Record

In [40]:
from eligibility_signposting_api.model.person import Person
from datetime import date
from dateutil.relativedelta import relativedelta

print("Creating Person Record")
print("=" * 60)
print()

# Calculate date of birth for 76 year old
today = date.today()
dob = today - relativedelta(years=76)

# Create person data (as it would come from DynamoDB)
person_dynamodb_items = [
    {
        "NHS_NUMBER": "9000000009",
        "ATTRIBUTE_TYPE": "PERSON",
        "DATE_OF_BIRTH": dob.strftime("%Y%m%d"),
        "POSTCODE": "LS1 4AP",
    },
    {
        "NHS_NUMBER": "9000000009",
        "ATTRIBUTE_TYPE": "COHORTS",
        "COHORT_MEMBERSHIPS": [
            {
                "COHORT_LABEL": "RSV_COHORT",
                "DATE_JOINED": "20240901"
            }
        ]
    },
    {
        "NHS_NUMBER": "9000000009",
        "ATTRIBUTE_TYPE": "RSV",  # Target condition as ATTRIBUTE_TYPE
        # No LAST_SUCCESSFUL_DATE means never vaccinated
        # If vaccinated, would have: "LAST_SUCCESSFUL_DATE": "20230415" (YYYYMMDD format)
    }
]

# Create Person object
person = Person(data=person_dynamodb_items)

print("Person created with attributes:")
for item in person_dynamodb_items:
    print(f"  {item['ATTRIBUTE_TYPE']}:")
    for key, value in item.items():
        if key not in ['NHS_NUMBER', 'ATTRIBUTE_TYPE']:
            print(f"    • {key}: {value}")
    print()

print("✓ Person record created")

Creating Person Record

Person created with attributes:
  PERSON:
    • DATE_OF_BIRTH: 19491014
    • POSTCODE: LS1 4AP

  COHORTS:
    • COHORT_MEMBERSHIPS: [{'COHORT_LABEL': 'RSV_COHORT', 'DATE_JOINED': '20240901'}]

  RSV:

✓ Person record created


## Creating an RSV Campaign Configuration

This campaign uses rules that reflect, but don't match, the actual production configuration.

Edd - show the actual config

- **Filter**: Age check using DATE_OF_BIRTH (must be 75+)
- **Filter**: Vaccination history check (exclude if vaccinated in last 25 years)
- **Suppression**: Shows message if already vaccinated
- **Redirect**: Leeds residents directed to contact GP

In [41]:
from eligibility_signposting_api.model.campaign_config import (
    CampaignConfig,
    Iteration,
    IterationCohort,
    IterationRule,
    AvailableAction,
    ActionsMapper,
    RuleType,
    RuleOperator,
    RuleAttributeLevel
)
from datetime import date, timedelta

print("Creating RSV Campaign Configuration")
print("=" * 60)
print()

# Define campaign dates and iteration
# Make sure campaign is "live" by including today's date
today = date.today()
campaign_start = today - timedelta(days=30)  # Started 30 days ago
campaign_end = today + timedelta(days=150)   # Ends in 150 days
iteration_date = campaign_start

print(f"Campaign period: {campaign_start} to {campaign_end}")
print(f"Today: {today} (campaign is LIVE)")
print()

# Create actions mapper
actions_mapper = ActionsMapper({
    "CONTACT_GP_LEEDS": AvailableAction(
        ActionType="CONTACT",
        ExternalRoutingCode="CONTACT_GP_RSV",
        ActionDescription="Contact your GP surgery to arrange RSV vaccination",
        UrlLink="https://www.nhs.uk/service-search/find-a-gp",
        UrlLabel="Find your GP"
    ),
    "INFO_RSV": AvailableAction(
        ActionType="INFO",
        ExternalRoutingCode="INFO_RSV",
        ActionDescription="Learn more about RSV vaccination",
        UrlLink="https://www.nhs.uk/conditions/vaccinations/rsv-vaccine",
        UrlLabel="Learn more"
    )
})

# Create campaign rules using IterationRule
# Note: Rules follow the actual config structure from production
rules = [
    # Filter rule: Age >= 75 (using DATE_OF_BIRTH with year operator)
    IterationRule(
        Name="Remove under 75 Years",
        Type=RuleType.filter,
        Priority=10,
        AttributeName="DATE_OF_BIRTH",
        Operator=RuleOperator.year_gt,  # Y> means "years greater than"
        Comparator="-75",  # -75 years from today
        AttributeTarget="",
        Description="Ensure anyone who has a DOB determining age < 75 is filtered out",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CohortLabel="RSV_COHORT",
        RuleStop="N"
    ),
    # Filter rule: Not already vaccinated (checks LAST_SUCCESSFUL_DATE)
    IterationRule(
        Name="Remove from RSV cohort if already vaccinated",
        Type=RuleType.filter,
        Priority=20,
        AttributeName="LAST_SUCCESSFUL_DATE",
        Operator=RuleOperator.year_gte,  # Y>= means "years greater than or equal"
        Comparator="-25",  # Vaccinated within last 25 years
        AttributeTarget="RSV",
        Description="Remove anyone already vaccinated from RSV cohort",
        AttributeLevel=RuleAttributeLevel.TARGET,
        CohortLabel="RSV_COHORT",
        RuleStop="N"
    ),
    # Suppression rule: Already vaccinated (with user-facing message)
    IterationRule(
        Name="AlreadyVaccinated",
        Type=RuleType.suppression,
        Priority=30,
        AttributeName="LAST_SUCCESSFUL_DATE",
        Operator=RuleOperator.year_gte,
        Comparator="-25",
        AttributeTarget="RSV",
        Description="## You've had your RSV vaccination\n\nOur records show you were vaccinated against RSV.",
        AttributeLevel=RuleAttributeLevel.TARGET,
        RuleStop="Y"  # Stop processing if matched
    ),
    # Redirect rule: Leeds area - contact GP
    IterationRule(
        Name="LeedsAreaContactGP",
        Type=RuleType.redirect,
        Priority=100,
        AttributeName="POSTCODE",
        Operator=RuleOperator.starts_with,
        Comparator="LS",  # Comparator is the VALUE to compare against
        AttributeTarget="",
        Description="Leeds residents should contact their GP",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CommsRouting="CONTACT_GP_LEEDS",
        RuleStop="N"
    )
]

# Create iteration with cohorts
iteration = Iteration(
    ID="RSV_2024_ITER_001",
    Version=1,
    Name="RSV_Winter_2024",
    IterationDate=iteration_date,
    Type="A",
    DefaultCommsRouting="INFO_RSV",
    DefaultNotEligibleRouting="INFO_RSV",
    DefaultNotActionableRouting="INFO_RSV",
    IterationCohorts=[
        IterationCohort(
            CohortLabel="RSV_COHORT",
            CohortGroup="Older Adults"
        )
    ],
    IterationRules=rules,
    ActionsMapper=actions_mapper
)

# Create campaign config
campaign_config = CampaignConfig(
    ID="RSV_WINTER_2024",
    Version=1,
    Name="RSV_WINTER_2024",
    Type="V",
    Target="RSV",
    IterationFrequency="A",
    IterationType="A",
    StartDate=campaign_start,
    EndDate=campaign_end,
    Iterations=[iteration]
)

print("Campaign Configuration Created:")
print(f"  Campaign: {campaign_config.name}")
print(f"  Condition: {campaign_config.target}")
print(f"  Period: {campaign_start} to {campaign_end}")
print(f"  Campaign Live: {campaign_config.campaign_live}")
print(f"  Rules: {len(rules)}")
print()
print("Rules:")
for rule in rules:
    print(f"  • {rule.name} ({rule.type.value}) - Priority {rule.priority}")
    print(f"    Check: {rule.attribute_name} {rule.operator.value} {rule.attribute_target}")
print()
print("Actions available:")
for code, action in actions_mapper.root.items():
    print(f"  • {code}: {action.action_description}")
print()
print("✓ Campaign configuration created")

Creating RSV Campaign Configuration

Campaign period: 2025-09-14 to 2026-03-13
Today: 2025-10-14 (campaign is LIVE)

Campaign Configuration Created:
  Campaign: RSV_WINTER_2024
  Condition: RSV
  Period: 2025-09-14 to 2026-03-13
  Campaign Live: True
  Rules: 4

Rules:
  • Remove under 75 Years (F) - Priority 10
    Check: DATE_OF_BIRTH Y> 
  • Remove from RSV cohort if already vaccinated (F) - Priority 20
    Check: LAST_SUCCESSFUL_DATE Y>= RSV
  • AlreadyVaccinated (S) - Priority 30
    Check: LAST_SUCCESSFUL_DATE Y>= RSV
  • LeedsAreaContactGP (R) - Priority 100
    Check: POSTCODE starts_with 

Actions available:
  • CONTACT_GP_LEEDS: Contact your GP surgery to arrange RSV vaccination
  • INFO_RSV: Learn more about RSV vaccination

✓ Campaign configuration created


---

# Part 3: Running the Eligibility Service

Now we'll run the actual eligibility calculation using our real service code.

## Create the Eligibility Calculator

In [42]:
from eligibility_signposting_api.services.calculators.eligibility_calculator import (
    EligibilityCalculatorFactory
)

print("Creating Eligibility Calculator")
print("=" * 60)
print()

# Create calculator with person and campaign data
calculator = EligibilityCalculatorFactory.get(
    person=person,
    campaign_configs=[campaign_config]
)

print("✓ Calculator created with:")
print(f"  • Person: NHS Number {person_dynamodb_items[0]['NHS_NUMBER']}")
print(f"  • Campaigns: {len([campaign_config])}")

Creating Eligibility Calculator

✓ Calculator created with:
  • Person: NHS Number 9000000009
  • Campaigns: 1


## Calculate Eligibility Status

In [43]:
from eligibility_signposting_api.model.eligibility_status import NHSNumber, ConditionName

print("Running Eligibility Calculation")
print("=" * 60)
print()

# Get eligibility status (same as what EligibilityService would do)
eligibility_status = calculator.get_eligibility_status(
    include_actions="Y",
    conditions=["RSV"],
    category="VACCINATIONS"
)

print("✓ Eligibility calculated")
print()
print("Processing what happened...")

Running Eligibility Calculation

✓ Eligibility calculated

Processing what happened...


## Examine the Results

In [44]:
print("Eligibility Calculation Results")
print("=" * 60)
print()

# Debug: Check what we got back
print(f"Number of conditions in result: {len(eligibility_status.conditions)}")
print(f"Eligibility status object: {eligibility_status}")
print()

# Check if we have any conditions
if not eligibility_status.conditions:
    print("❌ ERROR: No conditions returned from eligibility calculation")
    print("This might mean the campaign configuration doesn't match the requested condition.")
else:
    print(f"✓ Found {len(eligibility_status.conditions)} condition(s)")

print()
print("=" * 60)
print()

# Only proceed if we have conditions
if eligibility_status.conditions:
    # Get the RSV condition result
    rsv_condition = eligibility_status.conditions[0]

    print(f"Condition: {rsv_condition.condition_name}")
    print(f"Status: {rsv_condition.status.name.upper()}")
    print(f"Status Text: {rsv_condition.status_text}")
    print()

    # Show which rules were evaluated (suitability_rules)
    print("Rules Evaluated:")
    for reason in rsv_condition.suitability_rules:
        matched_symbol = "✓" if reason.matcher_matched else "✗"
        matched_text = "MATCHED" if reason.matcher_matched else "NOT MATCHED"
        print(f"  {matched_symbol} {reason.rule_name} ({reason.rule_type.value}): {matched_text}")
        if reason.rule_description:
            print(f"      {reason.rule_description}")
    print()

    # Show suggested actions
    if rsv_condition.actions:
        print(f"Suggested Actions ({len(rsv_condition.actions)}):")
        for action in rsv_condition.actions:
            print(f"  • {action.action_type}: {action.action_description}")
            if action.url_link:
                print(f"    Link: {action.url_link}")
                if action.url_label:
                    print(f"    Label: {action.url_label}")
    else:
        print("No actions suggested")

    print()
    print("=" * 60)
    print()

    # Summary
    if rsv_condition.status.name == "actionable":
        print("✅ RESULT: Person is ELIGIBLE for RSV vaccination")
        print(f"   They should: {rsv_condition.actions[0].action_description if rsv_condition.actions else 'See details above'}")
    elif rsv_condition.status.name == "not_actionable":
        print("⚠️  RESULT: Person is eligible but NOT ACTIONABLE")
    else:
        print("❌ RESULT: Person is NOT ELIGIBLE for RSV vaccination")

Eligibility Calculation Results

Number of conditions in result: 1
Eligibility status object: EligibilityStatus(conditions=[Condition(condition_name='RSV', status=<Status.actionable: 3>, cohort_results=[CohortGroupResult(cohort_code='Older Adults', status=<Status.actionable: 3>, reasons=[], description='', audit_rules=[])], suitability_rules=[], status_text='You should have the RSV vaccine', actions=[SuggestedAction(action_type='CONTACT', action_code='CONTACT_GP_RSV', action_description='Contact your GP surgery to arrange RSV vaccination', url_link=HttpUrl('https://www.nhs.uk/service-search/find-a-gp'), url_label='Find your GP', internal_action_code='CONTACT_GP_LEEDS')])])

✓ Found 1 condition(s)


Condition: RSV
Status: ACTIONABLE
Status Text: You should have the RSV vaccine

Rules Evaluated:

Suggested Actions (1):
  • CONTACT: Contact your GP surgery to arrange RSV vaccination
    Link: https://www.nhs.uk/service-search/find-a-gp
    Label: Find your GP


✅ RESULT: Person is ELIGIBL

## JSON Response Format

This is what would be returned to the API caller:

In [45]:
import json

print("API Response (JSON):")
print("=" * 60)
print()

# Convert to dict (similar to what the view layer does)
# Note: In the actual API, this goes through additional response models
response_data = {
    "conditions": [
        {
            "condition": rsv_condition.condition_name,
            "status": rsv_condition.status.name,
            "statusText": rsv_condition.status_text,
            "reasons": [
                {
                    "ruleType": reason.rule_type.value,
                    "ruleName": reason.rule_name,
                    "matched": reason.matcher_matched,
                    "description": reason.rule_description
                }
                for reason in rsv_condition.suitability_rules
            ],
            "suggestedActions": [
                {
                    "actionType": action.action_type,
                    "actionCode": action.action_code,
                    "description": action.action_description,
                    "link": str(action.url_link) if action.url_link else None,
                    "linkLabel": action.url_label if action.url_label else None
                }
                for action in (rsv_condition.actions or [])
            ]
        }
    ]
}

print(json.dumps(response_data, indent=2))
print()
print("This would be returned as HTTP 200 with Content-Type: application/fhir+json")

API Response (JSON):

{
  "conditions": [
    {
      "condition": "RSV",
      "status": "actionable",
      "statusText": "You should have the RSV vaccine",
      "reasons": [],
      "suggestedActions": [
        {
          "actionType": "CONTACT",
          "actionCode": "CONTACT_GP_RSV",
          "description": "Contact your GP surgery to arrange RSV vaccination",
          "link": "https://www.nhs.uk/service-search/find-a-gp",
          "linkLabel": "Find your GP"
        }
      ]
    }
  ]
}

This would be returned as HTTP 200 with Content-Type: application/fhir+json


Some more realistic responses where we shaped the message using different content based on rules specified in the configuration

### Actionable due to cohort membership, where we suggest local booking / GP

In [46]:
url = "https://sandbox.api.service.nhs.uk/eligibility-signposting-api/patient-check/9686368906"
headers = {"apiKey": "test"}

response = requests.get(url, headers=headers)
print("Status Code:", response.status_code)
print()
print("Response Body:")
print("=" * 60)

# Try to parse and prettify JSON response
try:
    response_json = response.json()
    print(json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    # If not JSON, print as text
    print(response.text)

Status Code: 200

Response Body:
{
  "responseId": "1a233ba5-e1eb-4080-a086-2962f6fc3473",
  "meta": {
    "lastUpdated": "2025-02-12T16:11:22Z"
  },
  "processedSuggestions": [
    {
      "condition": "RSV",
      "status": "Actionable",
      "statusText": "You should have the RSV vaccine",
      "eligibilityCohorts": [
        {
          "cohortCode": "rsv_age_rolling",
          "cohortText": "are aged 75 to 79",
          "cohortStatus": "Actionable"
        }
      ],
      "suitabilityRules": [],
      "actions": [
        {
          "actionType": "InfoText",
          "actionCode": "BookLocal",
          "description": "## Getting the vaccine\n\nYou can get an RSV vaccination at your GP surgery.\n\nYour GP surgery may contact you about getting the RSV vaccine. This may be by letter, text, phone call, email or through the NHS App. You do not need to wait to be contacted before booking your vaccination."
        }
      ]
    }
  ]
}


### Actionable due to cohort membership, where we suggest national booking

In [47]:
url = "https://sandbox.api.service.nhs.uk/eligibility-signposting-api/patient-check/9686368973"
headers = {"apiKey": "test"}

response = requests.get(url, headers=headers)
print("Status Code:", response.status_code)
print()
print("Response Body:")
print("=" * 60)

# Try to parse and prettify JSON response
try:
    response_json = response.json()
    print(json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    # If not JSON, print as text
    print(response.text)

Status Code: 200

Response Body:
{
  "responseId": "1a233ba5-e1eb-4080-a086-2962f6fc3473",
  "meta": {
    "lastUpdated": "2025-02-12T16:11:22Z"
  },
  "processedSuggestions": [
    {
      "condition": "RSV",
      "status": "Actionable",
      "statusText": "You should have the RSV vaccine",
      "eligibilityCohorts": [
        {
          "cohortCode": "rsv_age_rolling",
          "cohortText": "are aged 75 to 79",
          "cohortStatus": "Actionable"
        }
      ],
      "suitabilityRules": [],
      "actions": [
        {
          "actionType": "InfoText",
          "actionCode": "BookLocal",
          "description": "## Get vaccinated at your GP surgery\n\nContact your GP surgery to book an appointment."
        },
        {
          "actionType": "ButtonWithAuthLinkWithInfoText",
          "actionCode": "BookNBSInfoText",
          "description": "## Book an appointment online at a pharmacy\n\nYou can book an appointment online at a pharmacy that offers the RSV vaccinat

---

# Example 2: Not Eligible Person (Already Vaccinated)

Now let's test someone who **already had their RSV vaccination** to see how 'filtering' works.

## Create a Vaccinated Person

In [52]:
print("Creating Person Record - Already Vaccinated")
print("=" * 60)
print()

# Calculate date of birth for 78 year old
today = date.today()
dob_vaccinated = today - relativedelta(years=78)

# This person was vaccinated 6 months ago
vaccination_date = today - relativedelta(months=6)

# Create person data with vaccination history
person_vaccinated_items = [
    {
        "NHS_NUMBER": "9000000010",
        "ATTRIBUTE_TYPE": "PERSON",
        "DATE_OF_BIRTH": dob_vaccinated.strftime("%Y%m%d"),
        "POSTCODE": "M1 1AE",  # Manchester area
    },
    {
        "NHS_NUMBER": "9000000010",
        "ATTRIBUTE_TYPE": "COHORTS",
        "COHORT_MEMBERSHIPS": [
            {
                "COHORT_LABEL": "RSV_COHORT",
                "DATE_JOINED": "20240901"
            }
        ]
    },
    {
        "NHS_NUMBER": "9000000010",
        "ATTRIBUTE_TYPE": "RSV",
        "LAST_SUCCESSFUL_DATE": vaccination_date.strftime("%Y%m%d"),  # Vaccinated 6 months ago
    }
]

# Create Person object
person_vaccinated = Person(data=person_vaccinated_items)

print("Person created with attributes:")
for item in person_vaccinated_items:
    print(f"  {item['ATTRIBUTE_TYPE']}:")
    for key, value in item.items():
        if key not in ['NHS_NUMBER', 'ATTRIBUTE_TYPE']:
            print(f"    • {key}: {value}")
    print()

print(f"✓ Person record created (vaccinated on {vaccination_date.strftime('%Y-%m-%d')})")

Creating Person Record - Already Vaccinated

Person created with attributes:
  PERSON:
    • DATE_OF_BIRTH: 19471014
    • POSTCODE: M1 1AE

  COHORTS:
    • COHORT_MEMBERSHIPS: [{'COHORT_LABEL': 'RSV_COHORT', 'DATE_JOINED': '20240901'}]

  RSV:
    • LAST_SUCCESSFUL_DATE: 20250414

✓ Person record created (vaccinated on 2025-04-14)


## Calculate Eligibility for Vaccinated Person

In [53]:
print("Running Eligibility Calculation - Vaccinated Person")
print("=" * 60)
print()

# Create calculator with vaccinated person and same campaign config
calculator_vaccinated = EligibilityCalculatorFactory.get(
    person=person_vaccinated,
    campaign_configs=[campaign_config]
)

# Get eligibility status
eligibility_status_vaccinated = calculator_vaccinated.get_eligibility_status(
    include_actions="Y",
    conditions=["RSV"],
    category="VACCINATIONS"
)

print("✓ Eligibility calculated")
print()

Running Eligibility Calculation - Vaccinated Person

✓ Eligibility calculated



## Examine Results - Not Eligible

In [54]:
print("Eligibility Calculation Results - Vaccinated Person")
print("=" * 60)
print()

if eligibility_status_vaccinated.conditions:
    rsv_condition_vaccinated = eligibility_status_vaccinated.conditions[0]

    print(f"Condition: {rsv_condition_vaccinated.condition_name}")
    print(f"Status: {rsv_condition_vaccinated.status.name.upper()}")
    print(f"Status Text: {rsv_condition_vaccinated.status_text}")
    print()

    # Show which rules were evaluated
    print("Rules Evaluated:")
    for reason in rsv_condition_vaccinated.suitability_rules:
        matched_symbol = "✓" if reason.matcher_matched else "✗"
        matched_text = "MATCHED" if reason.matcher_matched else "NOT MATCHED"
        print(f"  {matched_symbol} {reason.rule_name} ({reason.rule_type.value}): {matched_text}")
        if reason.rule_description:
            # Show first line of description (may be markdown)
            desc_first_line = reason.rule_description.split('\n')[0]
            print(f"      {desc_first_line}")
    print()

    # Show suggested actions
    if rsv_condition_vaccinated.actions:
        print(f"Suggested Actions ({len(rsv_condition_vaccinated.actions)}):")
        for action in rsv_condition_vaccinated.actions:
            print(f"  • {action.action_type}: {action.action_description}")
            if action.url_link:
                print(f"    Link: {action.url_link}")
    else:
        print("No actions suggested (person suppressed from campaign)")

    print()
    print("=" * 60)
    print()

    # Summary
    if rsv_condition_vaccinated.status.name == "not_eligible":
        print("❌ RESULT: Person is NOT ELIGIBLE")
        print("   Reason: Already vaccinated - suppressed from campaign")
        if rsv_condition_vaccinated.actions:
            print(f"   Default action provided: {rsv_condition_vaccinated.actions[0].action_type}")
    elif rsv_condition_vaccinated.status.name == "not_actionable":
        print("⚠️  RESULT: Person is NOT ACTIONABLE")
    else:
        print("✅ RESULT: Person is ELIGIBLE")
else:
    print("❌ No conditions returned")

Eligibility Calculation Results - Vaccinated Person

Condition: RSV
Status: NOT_ELIGIBLE
Status Text: We do not believe you can have it

Rules Evaluated:

Suggested Actions (1):
  • INFO: Learn more about RSV vaccination
    Link: https://www.nhs.uk/conditions/vaccinations/rsv-vaccine


❌ RESULT: Person is NOT ELIGIBLE
   Reason: Already vaccinated - suppressed from campaign
   Default action provided: INFO


## A more realistic examples from the Sandbox

In [55]:
url = "https://sandbox.api.service.nhs.uk/eligibility-signposting-api/patient-check/9657933617"
headers = {"apiKey": "test"}

response = requests.get(url, headers=headers)
print("Status Code:", response.status_code)
print()
print("Response Body:")
print("=" * 60)

# Try to parse and prettify JSON response
try:
    response_json = response.json()
    print(json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    # If not JSON, print as text
    print(response.text)

Status Code: 200

Response Body:
{
  "responseId": "1a233ba5-e1eb-4080-a086-2962f6fc3473",
  "meta": {
    "lastUpdated": "2025-02-12T16:11:22Z"
  },
  "processedSuggestions": [
    {
      "condition": "RSV",
      "status": "NotEligible",
      "statusText": "We do not believe you can have it",
      "eligibilityCohorts": [
        {
          "cohortCode": "rsv_age_rolling",
          "cohortText": "are not aged 75 to 79",
          "cohortStatus": "NotEligible"
        },
        {
          "cohortCode": "rsv_age_catchup",
          "cohortText": "did not turn 80 between 2nd September 2024 and 31st August 2025",
          "cohortStatus": "NotEligible"
        }
      ],
      "suitabilityRules": [],
      "actions": [
        {
          "actionType": "InfoText",
          "actionCode": "HealthcareProInfo",
          "description": "## If you think you need this vaccine\n\nSpeak to your healthcare professional if you think you should be offered this vaccination."
        }
      ]

---

# Example 3: Not Actionable Person (Suppressed with Message)

Now let's create a scenario where someone is **not actionable** - they pass filter rules but hit a suppression rule with a user-facing message. This will show when `suitabilityRules` gets populated!

## Modified Campaign - Without Vaccination Filter

To demonstrate `not_actionable`, we need to remove the filter rule that checks vaccination status, keeping only the suppression rule.

In [56]:
print("Creating Modified RSV Campaign - Not Actionable Scenario")
print("=" * 60)
print()

# Create rules WITHOUT the vaccination filter (Priority 20)
# This allows vaccinated people to pass filter checks but hit suppression
rules_not_actionable = [
    # Filter rule: Age >= 75
    IterationRule(
        Name="Remove under 75 Years",
        Type=RuleType.filter,
        Priority=10,
        AttributeName="DATE_OF_BIRTH",
        Operator=RuleOperator.year_gt,
        Comparator="-75",
        AttributeTarget="",
        Description="Ensure anyone who has a DOB determining age < 75 is filtered out",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CohortLabel="RSV_COHORT",
        RuleStop="N"
    ),
    # *** REMOVED FILTER RULE FOR VACCINATION CHECK ***
    # This means vaccinated people will pass filter stage
    
    # Suppression rule: Already vaccinated (with user-facing message)
    IterationRule(
        Name="AlreadyVaccinated",
        Type=RuleType.suppression,
        Priority=30,
        AttributeName="LAST_SUCCESSFUL_DATE",
        Operator=RuleOperator.year_gte,
        Comparator="-25",
        AttributeTarget="RSV",
        Description="## You've had your RSV vaccination\n\nOur records show you were vaccinated against RSV.",
        AttributeLevel=RuleAttributeLevel.TARGET,
        RuleStop="Y"
    ),
    # Redirect rule: Leeds area
    IterationRule(
        Name="LeedsAreaContactGP",
        Type=RuleType.redirect,
        Priority=100,
        AttributeName="POSTCODE",
        Operator=RuleOperator.starts_with,
        Comparator="LS",
        AttributeTarget="",
        Description="Leeds residents should contact their GP",
        AttributeLevel=RuleAttributeLevel.PERSON,
        CommsRouting="CONTACT_GP_LEEDS",
        RuleStop="N"
    )
]

# Create iteration with modified rules
iteration_not_actionable = Iteration(
    ID="RSV_2024_ITER_002",
    Version=1,
    Name="RSV_Winter_2024_NotActionable",
    IterationDate=iteration_date,
    Type="A",
    DefaultCommsRouting="INFO_RSV",
    DefaultNotEligibleRouting="INFO_RSV",
    DefaultNotActionableRouting="INFO_RSV",
    IterationCohorts=[
        IterationCohort(
            CohortLabel="RSV_COHORT",
            CohortGroup="Older Adults"
        )
    ],
    IterationRules=rules_not_actionable,
    ActionsMapper=actions_mapper
)

# Create campaign config
campaign_config_not_actionable = CampaignConfig(
    ID="RSV_WINTER_2024_NA",
    Version=1,
    Name="RSV_WINTER_2024_NotActionable",
    Type="V",
    Target="RSV",
    IterationFrequency="A",
    IterationType="A",
    StartDate=campaign_start,
    EndDate=campaign_end,
    Iterations=[iteration_not_actionable]
)

print("Modified Campaign Configuration Created:")
print(f"  Difference: Removed filter rule for vaccination check")
print(f"  Rules: {len(rules_not_actionable)} (vs {len(rules)} in original)")
print()
print("Rules:")
for rule in rules_not_actionable:
    print(f"  • {rule.name} ({rule.type.value}) - Priority {rule.priority}")
print()
print("✓ Modified campaign configuration created")

Creating Modified RSV Campaign - Not Actionable Scenario

Modified Campaign Configuration Created:
  Difference: Removed filter rule for vaccination check
  Rules: 3 (vs 4 in original)

Rules:
  • Remove under 75 Years (F) - Priority 10
  • AlreadyVaccinated (S) - Priority 30
  • LeedsAreaContactGP (R) - Priority 100

✓ Modified campaign configuration created


## Calculate Eligibility with Modified Campaign

In [57]:
print("Running Eligibility Calculation - Not Actionable Scenario")
print("=" * 60)
print()

# Use the same vaccinated person from Example 2
# but with the modified campaign that has no vaccination filter
calculator_not_actionable = EligibilityCalculatorFactory.get(
    person=person_vaccinated,
    campaign_configs=[campaign_config_not_actionable]
)

# Get eligibility status
eligibility_status_not_actionable = calculator_not_actionable.get_eligibility_status(
    include_actions="Y",
    conditions=["RSV"],
    category="VACCINATIONS"
)

print("✓ Eligibility calculated with modified campaign")
print()

Running Eligibility Calculation - Not Actionable Scenario

✓ Eligibility calculated with modified campaign



## Examine Results - Not Actionable

In [58]:
print("Eligibility Calculation Results - Not Actionable")
print("=" * 60)
print()

if eligibility_status_not_actionable.conditions:
    rsv_condition_not_actionable = eligibility_status_not_actionable.conditions[0]

    print(f"Condition: {rsv_condition_not_actionable.condition_name}")
    print(f"Status: {rsv_condition_not_actionable.status.name.upper()}")
    print()

    # Show which rules were evaluated
    print("Rules Evaluated (suitability_rules):")
    if rsv_condition_not_actionable.suitability_rules:
        for reason in rsv_condition_not_actionable.suitability_rules:
            matched_symbol = "✓" if reason.matcher_matched else "✗"
            matched_text = "MATCHED" if reason.matcher_matched else "NOT MATCHED"
            print(f"  {matched_symbol} {reason.rule_name} ({reason.rule_type.value}): {matched_text}")
            if reason.rule_description:
                # Show first line of description
                desc_first_line = reason.rule_description.split('\n')[0]
                print(f"      {desc_first_line}")
    else:
        print("  (empty)")
    print()

    # Show suggested actions
    if rsv_condition_not_actionable.actions:
        print(f"Suggested Actions ({len(rsv_condition_not_actionable.actions)}):")
        for action in rsv_condition_not_actionable.actions:
            print(f"  • {action.action_type}: {action.action_description}")
            if action.url_link:
                print(f"    Link: {action.url_link}")
    else:
        print("No actions suggested")

    print()
    print("=" * 60)
    print()

    # Summary
    if rsv_condition_not_actionable.status.name == "not_actionable":
        print("⚠️  RESULT: Person is NOT ACTIONABLE")
        print("   Reason: Passed filter rules but suppressed with message")
        print(f"   Suitability rules populated: {len(rsv_condition_not_actionable.suitability_rules)} rule(s)")
    elif rsv_condition_not_actionable.status.name == "not_eligible":
        print("❌ RESULT: Person is NOT ELIGIBLE")
    else:
        print("✅ RESULT: Person is ELIGIBLE")
else:
    print("❌ No conditions returned")

Eligibility Calculation Results - Not Actionable

Condition: RSV
Status: NOT_ACTIONABLE

Rules Evaluated (suitability_rules):
  ✓ AlreadyVaccinated (S): MATCHED
      ## You've had your RSV vaccination

Suggested Actions (1):
  • INFO: Learn more about RSV vaccination
    Link: https://www.nhs.uk/conditions/vaccinations/rsv-vaccine


⚠️  RESULT: Person is NOT ACTIONABLE
   Reason: Passed filter rules but suppressed with message
   Suitability rules populated: 1 rule(s)


## Some more realistic examples from Sandbox

### Not actionable as dose not due yet

In [59]:
url = "https://sandbox.api.service.nhs.uk/eligibility-signposting-api/patient-check/9658219012"
headers = {"apiKey": "test"}

response = requests.get(url, headers=headers)
print("Status Code:", response.status_code)
print()
print("Response Body:")
print("=" * 60)

# Try to parse and prettify JSON response
try:
    response_json = response.json()
    print(json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    # If not JSON, print as text
    print(response.text)

Status Code: 200

Response Body:
{
  "responseId": "1a233ba5-e1eb-4080-a086-2962f6fc3473",
  "meta": {
    "lastUpdated": "2025-02-12T16:11:22Z"
  },
  "processedSuggestions": [
    {
      "condition": "RSV",
      "status": "NotActionable",
      "statusText": "You should have the RSV vaccine",
      "eligibilityCohorts": [
        {
          "cohortCode": "rsv_age_rolling",
          "cohortText": "are aged 75 to 79",
          "cohortStatus": "NotActionable"
        }
      ],
      "suitabilityRules": [
        {
          "ruleType": "S",
          "ruleCode": "NotYetDue",
          "ruleText": "Your next dose is not yet due."
        }
      ],
      "actions": []
    }
  ]
}


### Not actionable as vaccination given in other setting

In [60]:
url = "https://sandbox.api.service.nhs.uk/eligibility-signposting-api/patient-check/9658220150"
headers = {"apiKey": "test"}

response = requests.get(url, headers=headers)
print("Status Code:", response.status_code)
print()
print("Response Body:")
print("=" * 60)

# Try to parse and prettify JSON response
try:
    response_json = response.json()
    print(json.dumps(response_json, indent=2))
except json.JSONDecodeError:
    # If not JSON, print as text
    print(response.text)

Status Code: 200

Response Body:
{
  "responseId": "1a233ba5-e1eb-4080-a086-2962f6fc3473",
  "meta": {
    "lastUpdated": "2025-02-12T16:11:22Z"
  },
  "processedSuggestions": [
    {
      "condition": "RSV",
      "status": "NotActionable",
      "statusText": "You should have the RSV vaccine",
      "eligibilityCohorts": [
        {
          "cohortCode": "rsv_age_rolling",
          "cohortText": "are aged 75 to 79",
          "cohortStatus": "NotActionable"
        }
      ],
      "suitabilityRules": [
        {
          "ruleType": "S",
          "ruleCode": "OtherSetting",
          "ruleText": "## Getting the vaccine\n\nWe believe you're living in a setting where care is provided.\n\nSpeak to a member of staff where you live about getting the RSV vaccine."
        }
      ],
      "actions": []
    }
  ]
}


## Audit Records

The system creates detailed audit records for each eligibility check. Let's examine what was logged:

In [62]:
print("Complete Audit Record Structure (JSON)")
print("=" * 60)
print()
# Access the audit log from Flask's g object
audit_record = g.audit_log
# Convert audit record to dict for JSON serialization
# Using model_dump to get the full structure
audit_dict = audit_record.model_dump(mode='json', exclude_none=False)

# Pretty print the JSON structure
print(json.dumps(audit_dict, indent=2, default=str))

print()
print("=" * 60)
print()
print("✓ This shows the complete internal audit structure")
print("✓ Note: request and response are nested objects")
print("✓ All rule evaluations are captured for compliance")

Complete Audit Record Structure (JSON)

{
  "request": {
    "request_timestamp": "2025-10-14T11:37:43.880417Z",
    "headers": {
      "x_request_id": null,
      "x_correlation_id": null,
      "nhsd_end_user_organisation_ods": null,
      "nhsd_application_id": null
    },
    "query_params": {
      "category": null,
      "conditions": null,
      "include_actions": null
    },
    "nhs_number": null
  },
  "response": {
    "response_id": null,
    "last_updated": null,
    "condition": [
      {
        "campaign_id": "RSV_WINTER_2024",
        "campaign_version": 1,
        "iteration_id": "RSV_2024_ITER_001",
        "iteration_version": 1,
        "condition_name": "RSV",
        "status": "actionable",
        "status_text": "You should have the RSV vaccine",
        "eligibility_cohorts": [
          {
            "cohort_code": "RSV_COHORT",
            "cohort_status": "actionable"
          }
        ],
        "eligibility_cohort_groups": [
          {
            "coho